In [1]:
import json
import numpy as np
import pandas as pd
import PyPDF2
import docx2txt
import sys
import matplotlib.pyplot as plt
%matplotlib inline 
import networkx as nx
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from pyrouge import Rouge155
from nltk.tokenize.treebank import TreebankWordDetokenizer
from scipy.sparse.linalg import svds
from rouge import Rouge 

In [2]:
%%time

data = []
with open('all.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

Wall time: 1.93 s


In [3]:
type(data)

list

In [4]:
import sys
pd.set_option("display.max_colwidth", 0)
np.set_printoptions(threshold=sys.maxsize)

In [5]:
df = pd.DataFrame(data)
df

category  \
0      tajuk utama   
1      teknologi     
2      hiburan       
3      tajuk utama   
4      tajuk utama   
...            ...   
18769  teknologi     
18770  showbiz       
18771  tajuk utama   
18772  tajuk utama   
18773  showbiz       

                                                                                                                                                                                                                                                                                                             gold_labels  \
0      [[False, True], [True, True], [False, False, False], [False, False], [False, False], [False, False], [False, False], [False], [False, False]]                                                                                                                                                                       
1      [[False, False, False, False], [False, True, True], [False, False, True], [False, False, False, False], [False, False], [False, False, False], [False, False], [False, False], [False, False, False], [False, False, False], [False, False, False], [False, False, False, False], [False, False], [False, False]]   
2      [[True], [True], [False, False], [False], [False], [False], [False], [False, False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False, False], [False, False]]                                                                                             
3      [[True, True], [False, False, False], [True], [False, False], [False, False, False]]                                                                                                                                                                                                                                
4      [[False, True], [True, True, True], [False], [False, False], [False, False], [False], [False, False]]                                                                                                                                                                                                               
...                                                                                                      ...                                                                                                                                                                                                               
18769  [[True, False], [False, True, True], [False, True, False, False], [False, False, False], [True, False, False, False], [False, False, False], [False, False], [False], [False, False]]                                                                                                                               
18770  [[True, True], [True], [False, False, False, False, False], [False], [False], [False], [False, False], [False, False, False, False], [False], [False, False], [False, False, False, False], [False, False, False]]                                                                                                  
18771  [[True, True], [False, True], [False], [False, False], [False, False, False, False, False], [False, False], [False, False], [False, False], [False, False], [False, False], [False, False]]                                                                                                                         
18772  [[True], [False], [False, False], [False], [False, False], [False], [False], [False, False], [False], [False]]                                                                                                                                                                                                      
18773  [[True, False], [False, False], [False, False, False, False, False, False], [True, True], [False, False, False, False, False, False, False, False], [False, False, False], [False, False, False, False, False], [False]]                                              

In [6]:
%%time

def low_rank_svd(matrix, singular_count):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

r_scores = []

for i in range(len(df)):
    article = df.paragraphs[i]
    ref_sum = df.summary[i]
    
    sentences_list = []
    for paragraph in article:
        for sentence in paragraph:
            sentences_list.append(TreebankWordDetokenizer().detokenize(sentence))
    
    ref_sentences_list =[]
    for sentence in ref_sum:
        ref_sentences_list.append(TreebankWordDetokenizer().detokenize(sentence))
        
    tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
    dt_matrix = tv.fit_transform(sentences_list)
    dt_matrix = dt_matrix.toarray()
    
    vocab = tv.get_feature_names()
    td_matrix = dt_matrix.T
    
    similarity_matrix = np.matmul(dt_matrix, dt_matrix.T)
    
    similarity_graph = nx.from_numpy_array(similarity_matrix)
    
    scores = nx.pagerank(similarity_graph)
    ranked_sentences = sorted(((score, index) for index, score
                          in scores.items()),
                         reverse=True)
    
    num_sentences = len(sentences_list)
    
    top_sentence = [ranked_sentences[index][1]
               for index in range(num_sentences)]
    top_sentence.sort()
    
    sentence_array = sorted(((scores[i], s) for i, s in enumerate(sentences_list)), reverse=True)
    sentence_array = np.asarray(sentence_array)
    
    rank_max = float(sentence_array[0][0])
    rank_min = float(sentence_array[len(sentence_array) - 1][0])
    
    temp_array = []

    # if all sentences have equal ranks, means they are all the same
    # taking any sentence will give the summary, say the first sentence
    flag = 0
    if rank_max - rank_min == 0:
        temp_array.append(0)
        flag = 1
    
    # If the sentence has different ranks
    if flag != 1:
        for i in range(0, len(sentence_array)):
            temp_array.append((float(sentence_array[i][0]) - rank_min) / (rank_max - rank_min))
        
    threshold = (sum(temp_array) / len(temp_array)) + 0.3
    
    sentence_list = []
    if len(temp_array) > 1:
        for i in range(0, len(temp_array)):
            if temp_array[i] > threshold:
                sentence_list.append(sentence_array[i][1])
    else:
        sentence_list.append(sentence_array[0][1])
        
    if len(sentence_list)==0:
        sentence_list.append(sentence_array[0][1])
        
    gen_sum = ' '.join(map(str, sentence_list))
    ref_sum = ' '.join(map(str, ref_sentences_list))
    
    rouge = Rouge()
    r_score = rouge.get_scores(gen_sum, ref_sum)
    r_scores += r_score

Wall time: 3min 36s


In [7]:
r_scores

[{'rouge-1': {'r': 0.4772727272727273,
   'p': 0.4772727272727273,
   'f': 0.4772727222727273},
  'rouge-2': {'r': 0.38, 'p': 0.37254901960784315, 'f': 0.37623761876286643},
  'rouge-l': {'r': 0.45454545454545453,
   'p': 0.45454545454545453,
   'f': 0.45454544954545456}},
 {'rouge-1': {'r': 0.2222222222222222, 'p': 1.0, 'f': 0.363636360661157},
  'rouge-2': {'r': 0.18518518518518517, 'p': 1.0, 'f': 0.31249999736328127},
  'rouge-l': {'r': 0.2222222222222222, 'p': 1.0, 'f': 0.363636360661157}},
 {'rouge-1': {'r': 0.358974358974359,
   'p': 0.208955223880597,
   'f': 0.2641509387451051},
  'rouge-2': {'r': 0.14583333333333334,
   'p': 0.08235294117647059,
   'f': 0.10526315328170069},
  'rouge-l': {'r': 0.3076923076923077,
   'p': 0.1791044776119403,
   'f': 0.22641508968850135}},
 {'rouge-1': {'r': 0.45454545454545453,
   'p': 0.5319148936170213,
   'f': 0.4901960734621299},
  'rouge-2': {'r': 0.3389830508474576,
   'p': 0.3333333333333333,
   'f': 0.3361344487818657},
  'rouge-l': {'r

In [8]:
len(r_scores)

18774

In [9]:
f1=0
for i in range(len(r_scores)):
    #print(scores[i].get('rouge-l').get('f'))
    f1 += r_scores[i].get('rouge-l').get('f')
f1/len(r_scores)

0.3670884705376853

In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
#cosine_sim = cosine_similarity(dt_matrix, dt_matrix)